In [1]:
import pygame, sys
from pygame.locals import *
import os
import numpy
import matplotlib

import qiskit
from qiskit import *

matplotlib.use("Agg")

import matplotlib.pyplot as plt
import matplotlib.backends.backend_agg as agg

pygame.init()
level1 = pygame.image.load("../Images/level1.jpg") #credit: <a href="/free-image/blue-pixels-wallpaper-9124.html">Blue Pixels Wallpaper</a>
level1_bg = pygame.transform.scale(level1, (800, 600))

mx, my = pygame.mouse.get_pos()

pygame 2.0.1 (SDL 2.0.14, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
class sprite(pygame.sprite.Sprite):
    def __init__(self, pos_x, pos_y):
        super().__init__()
        self.textBox = pygame.Rect(0,0, 160, 30)
        self.hasDialogue = False
        self.dialogueFont = pygame.font.SysFont('imprintshadow', 20)
        self.faceRight = True
        self.faceLeft = False
        self.comment = ""
        self.xChange = 0
        self.yChange = 0
        self.left = []
        self.right = []
        self.up = []
        self.down = []
        
        # Right Initial Sprite load
        tempLoad = pygame.image.load('../Images/spriteSequence/final-0009.png')
        duckAni = pygame.transform.scale(tempLoad, (64, 64))
        self.right.append(duckAni)
        for i in range(10, 45):
            temp = pygame.image.load('../Images/spriteSequence/final-00{}.png'.format(i))
            duckAni = pygame.transform.scale(temp, (64, 64))
            self.right.append(duckAni)
            
            
        #Left Initial Sprite Load
        tempLoad = pygame.image.load('../Images/Duck_Left/final-0009.png')
        duckAni = pygame.transform.scale(tempLoad, (64, 64))
        self.left.append(duckAni)
        for i in range(10, 45):
            temp = pygame.image.load('../Images/Duck_Left/final-00{}.png'.format(i))
            duckAni = pygame.transform.scale(temp, (64, 64))
            self.left.append(duckAni)
        
        #Up Initial Sprite Load
        for i in range(1, 30):
            if i < 10:
                temp = pygame.image.load('../Images/Duck_Up/final-000{}.png'.format(i))
                duckAni = pygame.transform.scale(temp, (64, 64))
                self.up.append(duckAni)
            else:
                temp = pygame.image.load('../Images/Duck_Up/final-00{}.png'.format(i))
                duckAni = pygame.transform.scale(temp, (64, 64))
                self.up.append(duckAni)
        #Down Initial Sprite Load
        for i in range(1, 30):
            if i < 10:
                temp = pygame.image.load('../Images/Duck_Down/final-000{}.png'.format(i))
                duckAni = pygame.transform.scale(temp, (64, 64))
                self.down.append(duckAni)
            else:
                temp = pygame.image.load('../Images/Duck_Down/final-00{}.png'.format(i))
                duckAni = pygame.transform.scale(temp, (64, 64))
                self.down.append(duckAni)
            
        
        self.right_index = 0
        self.left_index = 0
        self.up_index = 0
        self.down_index = 0
        
        self.image = self.right[0]
        self.rect = self.image.get_rect()
    def RightPressed(self):
        self.image = self.right[self.right_index]
        if self.right_index >= len(self.right)-1:
            self.right_index = 0
        else:
            self.right_index += 1
    def LeftPressed(self):
        self.image = self.left[self.left_index]
        if self.left_index >= len(self.left)-1:
            self.left_index = 0
        else:
            self.left_index += 1
    def UpPressed(self):
        self.image = self.up[self.up_index]
        if self.up_index >= len(self.up)-1:
            self.up_index = 0
        else:
            self.up_index += 1
            
    def DownPressed(self):
        self.image = self.down[self.down_index]
        if self.down_index >= len(self.down)-1:
            self.down_index = 0
        else:
            self.down_index += 1
    
    def update(self, x, y, screen):
        self.rect.x = x
        self.rect.y = y
        
        
        if self.hasDialogue:
            point = self.rect.topright
            
            #Get Comment Rect
            textObj = self.dialogueFont.render(self.comment,1,(0,0,0))
            self.commentRect = textObj.get_rect()
            self.textBox.width = (self.commentRect.width + 20)
            self.textBox.height = (self.commentRect.height + 20)
            #prevents dialogue from dr. Qwack to be off screen... TODO: FIX FLUTTER 
            
            if self.faceRight:
                #position text box to stick with duck
                self.textBox.bottomleft = self.rect.topright
                #position text to stick inside textbox
                self.commentRect.x = (self.textBox.x + 10)
                self.commentRect.y = (self.textBox.y + 10)
            elif self.faceLeft:
                #position text box to stick with duck
                self.textBox.bottomright = self.rect.topleft
                #position text to stick inside textbox
                self.commentRect.x = (self.textBox.x + 10)
                self.commentRect.y = (self.textBox.y + 10)
            pygame.draw.rect(screen, (255, 255, 255), self.textBox, border_radius = 5)
            screen.blit(textObj, self.commentRect)
            
    def draw(self, screen):
        screen.blit(self.image, self.rect)

In [1]:
def level1_intro(x, y, xChange, yChange):
    running = True
    duck_button = pygame.Rect(0,0,64,64)
    jump = False
    fall = False
    leftActive = False
    rightActive = False
    upActive = False
    downActive = False
    inst = True
    commentIter = 0
    count = 0
    playerX = 200
    playerY = 200
    xChange = 0
    yChange = 0
    y = 450
    x = 100
    duck = sprite(x, y)
    bg_movement = 0
    width = 800

    while running:
        
        screen.fill((0,0,0))
        screen.blit(level1, (0,0))
        
        # Setup
        mx, my = pygame.mouse.get_pos()
        
        drQwackDialogue = ["It's time to learn about quantum circuits!", 
                           "These special circuits make up a quantum computer",
                       "And they allow us to perform elementary quantum computations on qubits",
                        "Circuits often have multiple qubits",
                        "And they always start out in the zero state",
                        "We can add gates and measurements to operate on the qubits",
                       "And this can change the state of any qubit to something else",
                        "If you ever want to go back, press *ESC*",
                       "To go to the next stage, press the *N* button"]
        #Instructions
        textScreen = pygame.Rect(45, 183, 743, 75)
    
        duck.comment = drQwackDialogue[commentIter]
        
        #Event Handling
        click = False
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False
                    
                # movement
                if event.key == pygame.K_LEFT:
                    xChange = -2.5
                    leftActive = True
                if event.key == pygame.K_RIGHT:
                    xChange = 2.5
                    rightActive = True
                if event.key == pygame.K_UP:
                    yChange = -2.5
                    upActive = True
                if event.key == pygame.K_DOWN:
                    yChange = 2.5
                    downActive = True
                if event.key == pygame.K_SPACE and commentIter < len(drQwackDialogue)+1:
                    commentIter += 1
                elif event.key == pygame.K_SPACE and commentIter == len(drQwackDialogue)+2:
                    inst = False
                    commentIter = 0
                elif event.key == pygame.K_n:
                    level1_example_circ(x, y, xChange, yChange)
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                    xChange = 0
                    rightActive = False
                    leftActive = False
                if event.key == pygame.K_UP or event.key == pygame.K_DOWN:
                    yChange = 0
                    upActive = False
                    downActive = False
            if event.type == MOUSEBUTTONDOWN:
                if event.button == 1:
                    click = True
        
        if commentIter >= len(drQwackDialogue):
            commentIter = 0
        else:
            duck.hasDialogue = True
            duck.comment = drQwackDialogue[commentIter]  
            
        #fix movement
        x += xChange
        if x <= 0:
            x = 0
        elif x >= 800-duck.rect.width:
            x = 800-duck.rect.width

        y += yChange
        if y <= 0:
            y = 0
        elif y >= 600-duck.rect.height:
            y = 600-duck.rect.height
                
        duck.hasDialogue = True
        if rightActive:
            duck.RightPressed()
        elif leftActive:
            duck.LeftPressed()
        elif upActive:
            duck.UpPressed()
        elif downActive:
            duck.DownPressed()
        duck.update(x, y, screen)
        
        #Display
        count += 1
        duck.draw(screen)
        pygame.display.update()
        mainClock.tick(120)